In [1]:
# This code just limits the gpu usage till 60% ...otherwise you will get memory error
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
def load_data(dataset_path):
    with open(dataset_path, "r") as fi :
        data = json.load(fi)
        
    # convert list into numpy array bcoz labels and mfccs are stored into lists
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])
    
    return inputs,targets

In [4]:
inputs, targets = load_data('data.json')

In [5]:
inputs.shape, targets.shape

((100, 130, 13), (100,))

In [6]:
# create train/validation/test data
# build CNN
# train the CNN
# evaluate the CNN on test set
# make prediction on sample

In [7]:
X, y = load_data('data.json')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

# now expand the dimensions bcoz CNN takes 4D tensors 
X_train = X_train[..., np.newaxis] # 4d array -> (num_samples, 130, 13, 1)
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [8]:
X_train.shape, X_validation.shape, X_test.shape

((60, 130, 13, 1), (15, 130, 13, 1), (25, 130, 13, 1))

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    tf.keras.layers.MaxPool2D((3,3), strides=(2,2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((3,3), strides=(2,2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(32, (2,2), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2), strides=(2,2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(10, activation='softmax')
    
    
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 128, 11, 32)       320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 6, 32)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 6, 32)         128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 62, 4, 32)         9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 2, 32)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 31, 2, 32)         128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 1, 32)        

In [34]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss = "sparse_categorical_crossentropy",
              metrics = ['accuracy'])
model.fit(X_train, y_train,
          validation_data=(X_validation,y_validation),
          batch_size=16,
          epochs=500)

test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy on the test set is {}".format(test_accuracy))

Train on 60 samples, validate on 15 samples
Epoch 1/500
60/60 [==============================] - 1s 15ms/sample - loss: 0.7281 - accuracy: 0.8167 - val_loss: 2.1398 - val_accuracy: 0.4000
Epoch 2/500
60/60 [==============================] - 0s 467us/sample - loss: 0.6180 - accuracy: 0.8667 - val_loss: 2.1176 - val_accuracy: 0.4000
Epoch 3/500
60/60 [==============================] - 0s 500us/sample - loss: 0.5717 - accuracy: 0.9333 - val_loss: 2.0768 - val_accuracy: 0.4000
Epoch 4/500
60/60 [==============================] - 0s 467us/sample - loss: 0.6163 - accuracy: 0.8500 - val_loss: 2.0331 - val_accuracy: 0.4000
Epoch 5/500
60/60 [==============================] - 0s 467us/sample - loss: 0.5397 - accuracy: 0.9167 - val_loss: 1.9817 - val_accuracy: 0.4667
Epoch 6/500
60/60 [==============================] - 0s 433us/sample - loss: 0.4005 - accuracy: 0.9333 - val_loss: 1.9290 - val_accuracy: 0.4667
Epoch 7/500
60/60 [==============================] - 0s 433us/sample - loss: 0.5420 - a

Epoch 57/500
60/60 [==============================] - 0s 483us/sample - loss: 0.0813 - accuracy: 1.0000 - val_loss: 0.9024 - val_accuracy: 0.7333
Epoch 58/500
60/60 [==============================] - 0s 467us/sample - loss: 0.0946 - accuracy: 1.0000 - val_loss: 0.8933 - val_accuracy: 0.7333
Epoch 59/500
60/60 [==============================] - 0s 500us/sample - loss: 0.0753 - accuracy: 1.0000 - val_loss: 0.8929 - val_accuracy: 0.8000
Epoch 60/500
60/60 [==============================] - 0s 466us/sample - loss: 0.0982 - accuracy: 1.0000 - val_loss: 0.8890 - val_accuracy: 0.8000
Epoch 61/500
60/60 [==============================] - 0s 466us/sample - loss: 0.0814 - accuracy: 1.0000 - val_loss: 0.8869 - val_accuracy: 0.8000
Epoch 62/500
60/60 [==============================] - 0s 483us/sample - loss: 0.1465 - accuracy: 1.0000 - val_loss: 0.8964 - val_accuracy: 0.7333
Epoch 63/500
60/60 [==============================] - 0s 467us/sample - loss: 0.0684 - accuracy: 1.0000 - val_loss: 0.8999 -

60/60 [==============================] - 0s 416us/sample - loss: 0.0572 - accuracy: 1.0000 - val_loss: 0.7507 - val_accuracy: 0.7333
Epoch 114/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0321 - accuracy: 1.0000 - val_loss: 0.7458 - val_accuracy: 0.7333
Epoch 115/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0370 - accuracy: 1.0000 - val_loss: 0.7427 - val_accuracy: 0.8000
Epoch 116/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0245 - accuracy: 1.0000 - val_loss: 0.7427 - val_accuracy: 0.8000
Epoch 117/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0418 - accuracy: 1.0000 - val_loss: 0.7414 - val_accuracy: 0.8000
Epoch 118/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0347 - accuracy: 1.0000 - val_loss: 0.7412 - val_accuracy: 0.8000
Epoch 119/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0305 - accuracy: 1.0000 - val_loss: 0.7393 - val_ac

Epoch 169/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0405 - accuracy: 1.0000 - val_loss: 0.7314 - val_accuracy: 0.7333
Epoch 170/500
60/60 [==============================] - 0s 450us/sample - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.7340 - val_accuracy: 0.7333
Epoch 171/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0161 - accuracy: 1.0000 - val_loss: 0.7354 - val_accuracy: 0.7333
Epoch 172/500
60/60 [==============================] - 0s 450us/sample - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.7369 - val_accuracy: 0.7333
Epoch 173/500
60/60 [==============================] - 0s 450us/sample - loss: 0.0163 - accuracy: 1.0000 - val_loss: 0.7369 - val_accuracy: 0.7333
Epoch 174/500
60/60 [==============================] - 0s 450us/sample - loss: 0.0295 - accuracy: 1.0000 - val_loss: 0.7425 - val_accuracy: 0.7333
Epoch 175/500
60/60 [==============================] - 0s 450us/sample - loss: 0.0239 - accuracy: 1.0000 - val_loss: 0

Epoch 225/500
60/60 [==============================] - 0s 467us/sample - loss: 0.0181 - accuracy: 1.0000 - val_loss: 0.6796 - val_accuracy: 0.8000
Epoch 226/500
60/60 [==============================] - 0s 451us/sample - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.6685 - val_accuracy: 0.8000
Epoch 227/500
60/60 [==============================] - 0s 450us/sample - loss: 0.0230 - accuracy: 1.0000 - val_loss: 0.6680 - val_accuracy: 0.8000
Epoch 228/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0144 - accuracy: 1.0000 - val_loss: 0.6679 - val_accuracy: 0.8000
Epoch 229/500
60/60 [==============================] - 0s 466us/sample - loss: 0.0169 - accuracy: 1.0000 - val_loss: 0.6764 - val_accuracy: 0.8000
Epoch 230/500
60/60 [==============================] - 0s 449us/sample - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.6841 - val_accuracy: 0.8000
Epoch 231/500
60/60 [==============================] - 0s 450us/sample - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0

Epoch 281/500
60/60 [==============================] - 0s 484us/sample - loss: 0.0104 - accuracy: 1.0000 - val_loss: 0.6655 - val_accuracy: 0.8000
Epoch 282/500
60/60 [==============================] - 0s 550us/sample - loss: 0.0094 - accuracy: 1.0000 - val_loss: 0.6705 - val_accuracy: 0.8000
Epoch 283/500
60/60 [==============================] - 0s 517us/sample - loss: 0.0149 - accuracy: 1.0000 - val_loss: 0.6775 - val_accuracy: 0.8000
Epoch 284/500
60/60 [==============================] - 0s 467us/sample - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.6835 - val_accuracy: 0.8000
Epoch 285/500
60/60 [==============================] - 0s 449us/sample - loss: 0.0091 - accuracy: 1.0000 - val_loss: 0.6883 - val_accuracy: 0.8000
Epoch 286/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.6911 - val_accuracy: 0.8667
Epoch 287/500
60/60 [==============================] - 0s 400us/sample - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0

60/60 [==============================] - 0s 416us/sample - loss: 0.0104 - accuracy: 1.0000 - val_loss: 0.6023 - val_accuracy: 0.8667
Epoch 337/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.6049 - val_accuracy: 0.8667
Epoch 338/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.6073 - val_accuracy: 0.8667
Epoch 339/500
60/60 [==============================] - 0s 434us/sample - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.6124 - val_accuracy: 0.8000
Epoch 340/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0196 - accuracy: 1.0000 - val_loss: 0.6255 - val_accuracy: 0.8000
Epoch 341/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.6407 - val_accuracy: 0.8000
Epoch 342/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.6553 - val_ac

60/60 [==============================] - 0s 433us/sample - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.6872 - val_accuracy: 0.8000
Epoch 392/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.6862 - val_accuracy: 0.8000
Epoch 393/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0111 - accuracy: 1.0000 - val_loss: 0.6867 - val_accuracy: 0.8000
Epoch 394/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.6819 - val_accuracy: 0.8000
Epoch 395/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0138 - accuracy: 1.0000 - val_loss: 0.6746 - val_accuracy: 0.8000
Epoch 396/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0066 - accuracy: 1.0000 - val_loss: 0.6709 - val_accuracy: 0.8000
Epoch 397/500
60/60 [==============================] - 0s 400us/sample - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.6680 - val_ac

Epoch 447/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.6328 - val_accuracy: 0.8000
Epoch 448/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.6408 - val_accuracy: 0.8000
Epoch 449/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.6467 - val_accuracy: 0.8000
Epoch 450/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.6475 - val_accuracy: 0.8000
Epoch 451/500
60/60 [==============================] - 0s 416us/sample - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.6478 - val_accuracy: 0.8000
Epoch 452/500
60/60 [==============================] - 0s 433us/sample - loss: 0.0119 - accuracy: 1.0000 - val_loss: 0.6435 - val_accuracy: 0.8000
Epoch 453/500
60/60 [==============================] - 0s 417us/sample - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0

In [29]:
X  = X_test[10]
X.shape

(130, 13, 1)

In [30]:
# predictions

X  = X_test[10]
X = X[np.newaxis, ...]
y = y_test[10]


prediction = model.predict(X, y) # convert X to 3d to 4d then predict.. coz conv takes 4d 
prediction # represents 10 values from 10 labels

array([[0.04745073, 0.00374796, 0.03951628, 0.06148709, 0.02907085,
        0.01512958, 0.6117592 , 0.02583513, 0.07398392, 0.09201939]],
      dtype=float32)

In [31]:
X.shape

(1, 130, 13, 1)

In [27]:
np.argmax(prediction)

6

In [28]:
y

6